In [1]:
#libraries
import cv2
import numpy as np
import pandas as pd
from sklearn.neighbors import KDTree
import json
import xml.etree.ElementTree as ET
from tqdm import tqdm
import zipfile
import os

In [31]:
# read the .xml-file
# zip_ref = zipfile.ZipFile('../CompanyMen_v1.0-split-012-Bobby_being_angry.azp')
zip_ref = zipfile.ZipFile('her_scene11_fuerChristian.azp')
zip_ref.extractall('/tmp')
tree = ET.parse('/tmp/content.xml')
root = tree.getroot().findall('./{http://experience.univ-lyon1.fr/advene/ns}annotations')
i=0
ts_end=[]
ts_begin=[]
for child in root[0].iter():
    if child.get('type')=='#Shot':
        i+=1
        for child2 in child:
            if child2.tag=='{http://experience.univ-lyon1.fr/advene/ns}millisecond-fragment':
                ts_end.append(round(int(child2.get('end'))/1000*25))
                ts_begin.append(round(int(child2.get('begin'))/1000*25))

In [32]:
def read_target_colors_azp(azp_path):
    zip_ref = zipfile.ZipFile(azp_path)
    zip_ref.extractall('/tmp')
    tree = ET.parse('/tmp/content.xml')
    root = tree.getroot().findall('./{http://experience.univ-lyon1.fr/advene/ns}annotations')
    colors_target=[]
    for child in root[0].iter():
        if child.get('type')=='#ColourRange':
            for child2 in child:
                if child2.tag=='{http://experience.univ-lyon1.fr/advene/ns}content':
                    colors_target.append(child2.text.split(','))
    return colors_target

In [33]:
print('ts_begin: ',np.shape(ts_begin),ts_begin)
print('ts_end: ',np.shape(ts_end),ts_end)

ts_begin:  (23,) [0, 239, 465, 4436, 4510, 4602, 4651, 4717, 4813, 4930, 4974, 5017, 5247, 5278, 5546, 5643, 5684, 5730, 5795, 5846, 5953, 6008, 6043]
ts_end:  (23,) [239, 465, 4436, 4510, 4602, 4651, 4717, 4813, 4930, 4974, 5017, 5247, 5278, 5546, 5643, 5684, 5730, 5795, 5846, 5953, 6008, 6043, 6387]


In [34]:
#read video file frame by frame, beginning and ending with a timestamp
def read_video_segments(video,start_frame,end_frame,resolution_width,target_colorspace):
    resolution_height=int(round(resolution_width * 9/16))
    resolution=(resolution_width,resolution_height)
    vid = cv2.VideoCapture(video)
    frames=[]
    vid_length=0
    with tqdm(total=end_frame-start_frame+1) as pbar: #init the progressbar,with max lenght of the given segment
        while(vid.isOpened()):
            # Capture frame-by-frame
            ret, frame = vid.read() # if ret is false, frame has no content
            if not ret:
                break
            # skip every "skip_frame"
            if vid_length>=start_frame:
                # resize the video to a different resolution
                frame=cv2.resize(frame,resolution)
                frame=np.array(frame,dtype='uint8')
                frames.append(frame) #add the individual frames to a list
                pbar.update(1) #update the progressbar
            if vid_length==end_frame:
                pbar.update(1)
                break
            vid_length+=1 #increase the vid_length counter
    vid.release()
    cv2.destroyAllWindows()
    frames=change_colorspace(frames,target_colorspace)
    return frames[:-1]

In [35]:
def change_colorspace(frame_list,target_colorspace):
    changed_frame_list=[]
    if target_colorspace=='HSV':
        print('HSV')
        for frame in frame_list:
            changed_frame_list.append(cv2.cvtColor(frame, cv2.COLOR_BGR2HSV))
        return changed_frame_list
    if target_colorspace=='cie-lab':
        print('cie-lab')
        for frame in frame_list:
            changed_frame_list.append(cv2.cvtColor(frame, cv2.COLOR_BGR2LAB))
        return changed_frame_list
    else:
        print('rgb')
        for frame in frame_list:
            changed_frame_list.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        return changed_frame_list

In [36]:
def fn_rgb_to_color(target_colorspace,path):
            if (path != 'full'):
                print('Now using colors specified in path')
                colors = {}
                with open(path) as f:
                    for line in f:
                        #split lines at "::
                        color, rgb = line.strip().split(':')
                        #strip the rgb-string of the parenthesis, split it up a the commas,
                        #cast them to int and put them into a tuples
                        rgb_value=tuple(map(int,(rgb.strip('(').strip(')').split(','))))
                        colors[color]=rgb_value
            else:
                colors={'darkred':(139,0,0),
                'firebrick':(178,34,34),
                'crimson':(220,20,60),
                'red':(255,0,0),
                'tomato':(255,99,71),
                'salmon':(250,128,114),
                'darkorange':(255,140,0),
                'gold':(255,215,0),
                'darkkhaki':(189,183,107),
                'yellow':(255,255,0),
                'darkolivegreen':(85,107,47),
                'olivedrab':(107,142,35),
                'greenyellow':(173,255,47),
                'darkgreen':(0,100,0),
                'aquamarine':(127,255,212),
                'steelblue':(70,130,180),
                'skyblue':(135,206,235),
                'darkblue':(0,0,139),
                'blue':(0,0,255),
                'royalblue':(65,105,225),
                'purple':(128,0,128),
                'violet':(238,130,238),
                'deeppink':(255,20,147),
                'pink':(255,192,203),
                'antiquewhite':(250,235,215),
                'saddlebrown':(139,69,19),
                'sandybrown':(244,164,96),
                'ivory':(255,255,240),
                'dimgrey':(105,105,105),
                'grey':(128,128,128),
                'silver':(192,192,192),
                'lightgrey':(211,211,211),
                'black':(0,0,0),
                'white':(255,255,255),
                'darkcyan':(0,139,139),
                'cyan':(0,255,255),
                'green':(0,128,0),
                'khaki':(240,230,140),
                'goldenrod':(218,165,32),
                'orange':(255,165,0),
                'coral':(255,127,80),
                'magenta':(255,0,255),
                'wheat':(245,222,179),
                'skin':(255,224,189),
                'purple4':(147,112,219)}

            colors_aux={}
            if target_colorspace=='HSV':
                print('HSV')
                for color in colors:
                    a = np.array((colors[color]),dtype='uint8')
                    b = a.reshape(1,1,3)
                    c = cv2.cvtColor(b,cv2.COLOR_RGB2HSV)
                    colors_aux[color]=tuple(c.reshape(3))
                colors=colors_aux
            if target_colorspace=='cie-lab':
                print('cie-lab')
                for color in colors:
                    a = np.array((colors[color]),dtype='uint8')
                    b = a.reshape(1,1,3)
                    c = cv2.cvtColor(b,cv2.COLOR_RGB2LAB)
                    colors_aux[color]=tuple(c.reshape(3))
                colors=colors_aux

            rgb_to_color={}
            for color in colors:
                rgb_to_color[colors[color]]=color
            #purple4 is median purple
            #skin is caucasian        
            return rgb_to_color

In [37]:
def create_nns_picture(frame_list,target_colorspace,path):
    rgb_to_color=fn_rgb_to_color(target_colorspace,path) #get the color dict 
    bins={} #a dict with an entry for each for histograms 
    for rgb in rgb_to_color: #init the dict with zeros for every key
        bins[rgb_to_color[rgb]]=0
        
    rgb_list=[] #create a list of the rgb_values
    for rgb in rgb_to_color: #map the values of the dict to a list
        rgb_list.append(rgb)
    kdt = KDTree(rgb_list, leaf_size=30, metric='euclidean')  
    #flatten the image to 1d 
#     img = frame_list[0].reshape((frame_list[0].shape[0] * frame_list[0].shape[1], 3))
    img = frame_list.reshape((frame_list.shape[0] * frame_list.shape[1], 3))     

    nns = kdt.query(img, k=1, return_distance=False)
    changed_frame_aux=[]
    for nn in tqdm(nns):
        changed_frame_aux.append(rgb_list[nn[0]])
    changed_frame_aux=np.asarray(changed_frame_aux,dtype='uint8')
#     changed_frame=changed_frame_aux.reshape(frame_list[0].shape[0],frame_list[0].shape[1],3)
    changed_frame=changed_frame_aux.reshape(frame_list.shape[0],frame_list.shape[1],3)
    return changed_frame

In [40]:
resolution=200
scene_all_frames_lab=[]
target_colorspace='cie-lab'
# color_path='../colors'
color_path='full'
# video_path='../../Wells_John_CompanyMen_full.mp4'
video_path='Her_bluray_Szene 11_25fps.mp4'
for i,ts in enumerate(zip(ts_begin,ts_end)):
    scene_all_frames_lab.append(read_video_segments(video_path,
                                                    ts[0],ts[1],resolution,target_colorspace))
#     if i==5:
#         break

241it [00:01, 189.67it/s]                        
  0%|          | 0/227 [00:00<?, ?it/s]

cie-lab


228it [00:02, 107.28it/s]                        
  0%|          | 0/3972 [00:00<?, ?it/s]

cie-lab


3973it [00:19, 203.14it/s]                          


...

cie-lab


  0%|          | 0/36 [00:00<?, ?it/s]

cie-lab



  0%|          | 0/345 [00:00<?, ?it/s]

cie-lab
cie-lab


In [41]:
print('shape: ', np.shape(scene_all_frames_lab))

shape:  (23,)


In [45]:
for i,frame_list in enumerate(scene_all_frames_lab):
#     print('full',len(frame_list))
#     print('half',int(len(frame_list)/2))
    index=int(len(frame_list)/2)
    try:
#         frame=frame_list[index]
        frame=create_nns_picture(frame_list[index],target_colorspace,color_path)
        cv2.imwrite('key_frames/ganzer_film/lab_full_200w_'+str(i)+'.png',cv2.cvtColor(frame, cv2.COLOR_LAB2BGR))
    except:
        print('no valid timestamps at '+str(i))
        pass

100%|██████████| 22400/22400 [00:00<00:00, 1671632.08it/s]

cie-lab
cie-lab
cie-lab
cie-lab
cie-lab
cie-lab
cie-lab



100%|██████████| 22400/22400 [00:00<00:00, 1726875.89it/s]


cie-lab
cie-lab
cie-lab
cie-lab
cie-lab
cie-lab
cie-lab


100%|██████████| 22400/22400 [00:00<00:00, 1984379.03it/s]

cie-lab
cie-lab
cie-lab
cie-lab
cie-lab
cie-lab



100%|██████████| 22400/22400 [00:00<00:00, 1757499.52it/s]

cie-lab
no valid timestamps at 21
no valid timestamps at 22


In [ ]:
def extract_dominant_colors(frame_list,target_colorspace,path):
    print(str(len(frame_list))+' frames to process.')
    rgb_to_color=fn_rgb_to_color(target_colorspace,path) #get the color dict 
    bins={} #bins dict for histograms 
    for rgb in rgb_to_color: #init the dict with ones for every key to avoid difficulties with divisions
                            # because the the sum of the bins goes from 500k to 2kk this shouldn't be a problem
        bins[rgb_to_color[rgb]]=1
    rgb_list=[] #create a traverseable list of the rgb_values
    for rgb in rgb_to_color: #map the values of the dict to a list
        rgb_list.append(rgb)

    kdt = KDTree(rgb_list, leaf_size=30, metric='euclidean')
    for image in tqdm(frame_list): #traverse the video
        img = image.reshape((image.shape[0] * image.shape[1], 3)) #flatten the image to 1d   
        nns = kdt.query(img, k=1, return_distance=False)
        for nn in nns:
            bins[rgb_to_color[rgb_list[nn[0]]]]+=1
    norm_factor = len(frame_list)* np.shape(frame_list[0])[0] * np.shape(frame_list[0])[1] #normalize the bins
    bins_norm={k:v/norm_factor*10000 for k,v in bins.items()} #scale 0-10000 for visibility
    return bins_norm

In [ ]:
def bins_to_df(bins,bin_threshold=5,colors_to_return=5):
    #create a dataframe
    bins_sorted=list(zip(list(bins.values()),list(bins.keys())))
    df=pd.DataFrame(bins_sorted,columns=['count','color'])
    df.set_index('color',inplace=True) #set the colors as the index of the dataframe
#     bin_threshold=bin_threshold/100 #scale the percentage to 0-1
#     df = df[df>bin_threshold].dropna() #kick bins from the dataframe with precentage lower than bin_threshold 
    return df

In [ ]:
# path='full'
dataframes=[]
for shot in scene_all_frames_lab:
    bins=extract_dominant_colors(shot,target_colorspace,color_path)
    dataframes.append(bins_to_df(bins))
#     break

In [ ]:
print(len(dataframes))

In [ ]:
def process_df(dataframe_list,ground_truth,noise_threshold):
    # create the lists that are returned
    # add the first entry of the histogramm to the output 
    # the 'real' output is the real_dataframe_list
    absolute_dataframe_list=[dataframe_list[0].sort_values(by='count',ascending=False)]
    relative_dataframe_list=[dataframe_list[0].sort_values(by='count',ascending=False)]
    real_dataframe_list=[dataframe_list[0].sort_values(by='count',ascending=False)]
    comparison=[dataframe_list[0].sort_values(by='count',ascending=False)]
    
    # traverse the histograms
    # d is the current shot, d1 the following shot
    for i,d in enumerate(zip(dataframe_list,dataframe_list[1:])):
        
        absolute_df=d[1]-d[0] #calculate the absolute change of the histograms
        absolute_df=absolute_df.sort_values(by='count',ascending=False)
        absolute_dataframe_list.append(absolute_df) #add the changes to the corresponding lists

        #apply a noise-filter to the absolute df
        absolute_high=absolute_df[absolute_df>noise_threshold].dropna()
        absolute_low=absolute_df[absolute_df<-noise_threshold].dropna()
        absolute_denoised=absolute_high.combine_first(absolute_low)
        
        # create the relative change, scaled to 0-100 percent
        # they are calculated from the denoised df, to prevent errors because of the noise
        relative_df=absolute_denoised/d[0]*100
        relative_df=relative_df.dropna()
        relative_df=relative_df.sort_values(by='count',ascending=False)
        relative_dataframe_list.append(relative_df)
        
        dd=d[0].sort_values(by='count',ascending=False)
        
        # if there are no elements in the denoised df it is assumed that all changes from
        # shot to shot are noise, in that case the current histogram is appended,
        # else the relative histogram
        if len(absolute_denoised)==0:
            real_dataframe_list.append(dd)
        else:
            real_dataframe_list.append(relative_df)
        
        new=relative_df.head()
        new=new.drop('count',axis=1)
        new=new.reset_index(level=0,inplace=False)
        new=new.rename(index=str,columns={'color':'new'})
        old=dd.head()
        old=old.drop('count',axis=1)
        old=old.reset_index(level=0,inplace=False)
        old=old.rename(index=str,columns={'color':'old'})

        gt=pd.DataFrame(ground_truth[i],columns=['ground_truth'])
        
        joined=pd.concat([old,new,gt],axis=1)
        comparison.append(joined)
#         print('len absolute_denoised: ',len(absolute_denoised))
        
    return {'real': real_dataframe_list,'comparison':comparison,'absolute':absolute_dataframe_list,'relative':relative_dataframe_list}

In [ ]:
ground_truth=read_target_colors_azp('her_scene11_fuerChristian.azp')
new = process_df(dataframes,ground_truth,100)
# print(len(new['real']))

In [ ]:
new['real'][3]

In [ ]:
new['comparison'][1]

In [ ]:
new['comparison'][2]

In [ ]:
new['comparison'][3]

In [ ]:
new['comparison'][4]

In [ ]:
new['comparison'][5]